This code runs the QME algorithm on a randomly generated $(a, B)$. QME uses LPME as a subroutine, so it effectively achieves its predictions using just oracle comparisons.

Note that the QME algorithm is more susceptible to error than LPME because it uses fractions. Keep following the code for an explanation. In the code we've built in an inconsistency checker that, when flipped on, checks the cosine similarity between expected (true) gradients and the gradients determined by LPME. Later in this notebook we run trials to see the error statistics, and **even in the worst case the gradient estimation is very accurate** with cosine similarity < $10^{-6}$. We show the error can be entirely attributed to the use of fractions, and we show comparisons to so-called "well-formed" $(a, B)$ that satisfy the regularity assumption in the paper to "ill-formed" $(a, B)$ that are truly random.

The trials run in this notebook will be saved to disk in *trials/qme/well_formed* and *trials/qme/ill_formed*. *qme_results_analyze.ipynb* will use these to make graphs.

In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import numpy as np
from scipy import spatial
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt

import sys
sys.path.append('../')
from common import Sphere, Oracle, create_a_B, normalize
from qme import QME, QMESC
from trials import NUM_TRIALS, load_qme_sphere, load_a_B, write_qme_trial, write_qme_trial_summary

# Example

In [3]:
np.random.seed(7)

# number of classes
nc = 2
# well_formed input
wf = True
# the tighter this is, the closer to the true answer (in all cases)
search_tol = 1e-3

q = nc ** 2 - nc

In [4]:
sphere = Sphere(np.zeros(q), 1.0, q)
a, B = create_a_B(sphere, q, wf)
oracle = Oracle(a, B)

In [5]:
qm = QME(sphere, oracle, search_tol, wf)
ahat, Bhat = qm.run_qme()

In [6]:
# not bad, not bad
print("a squared error:", np.linalg.norm(ahat - a))
print("B squared error:", np.linalg.norm(Bhat - B, ord='fro'))

a squared error: 0.0016076871061441085
B squared error: 0.0018994205472158427


# Trials

First run `python trials.py`.

Make sure to run all choices of $2 \leq nc \leq 5 $ and $wf \in \{True, False\}$. With 6-core multiprocessing, this should take 10-20 minutes total.

In [7]:
import multiprocessing as mp
import os

In [8]:
# -- Configurations --

# number of classes, change this as you go
nc = 2
# well-formed, change this as you go
wf = False

# set this based on your system
num_procs = 6

search_tol = 1e-2


# -- Vars --
q = nc ** 2 - nc
sphere = load_qme_sphere(nc, well_formed=wf)

folder = None
if wf:
    folder = "well_formed"
else:
    folder = "ill_formed"

if os.path.exists(f"trials/qme/{folder}/k={nc}/a_0_hat.npy"):
    print("WARNING - this class has already been run")

In [9]:
QUEUE_IN = mp.Queue()
QUEUE_OUT = mp.Queue()


def run_trial(sphere, a, B, search_tol, wf):
    oracle = Oracle(a, B)
    qm = QME(sphere, oracle, search_tol, wf)
    a_hat, B_hat = qm.run_qme()
    
    return a_hat, B_hat


def proc_run_trials(self_id, search_tol, sphere, wf):
    while True:
        data = QUEUE_IN.get(block=True)
        if data is None:
            QUEUE_IN.put(None) # so other threads can read this and exit out
            break # exit
            
        tid, a, B = data
        a_hat, B_hat = run_trial(sphere, a, B, search_tol, wf)
        
        # put result into queue out
        QUEUE_OUT.put((tid, a_hat, B_hat))

In [10]:
# start the procs
procs = []
for i in range(num_procs):
    proc = mp.Process(target=proc_run_trials, args=(
        i,
        search_tol,
        sphere,
        wf,
    ))
    proc.start()
    procs.append(proc)

In [11]:
# put in work
trial_ids = []
a_list = []
B_list = []

for i in tqdm_notebook(range(NUM_TRIALS)):
    a, B = load_a_B(nc, i, well_formed=wf)
    trial_ids.append(i)
    a_list.append(a)
    B_list.append(B)
    
    QUEUE_IN.put((i, a, B))
    
QUEUE_IN.put(None) # signal end to procs

C:\Users\Garfield_hant\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/300 [00:00<?, ?it/s]

In [ ]:
# use trial_ids_out to map into the original inputs
trial_ids_out = []

a_hat_list = []
B_hat_list = []

a_err = []
B_err = []

# we should get trials many results from QUEUE_OUT
for _ in tqdm_notebook(range(NUM_TRIALS)):
    tid, a_hat, B_hat = QUEUE_OUT.get(block=True)
    
    trial_ids_out.append(tid)
    
    a_hat_list.append(a_hat)
    B_hat_list.append(B_hat)
    
    # compute error
    a_err.append( np.linalg.norm(a_hat - a_list[tid]) )
    B_err.append( np.linalg.norm(B_hat - B_list[tid], ord='fro') )

C:\Users\Garfield_hant\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


  0%|          | 0/300 [00:00<?, ?it/s]

In [ ]:
a_err = np.array(a_err)
B_err = np.array(B_err)

In [ ]:
# save each trial result
for i in range(NUM_TRIALS):
    write_qme_trial(
        nc,
        trial_ids_out[i],
        a_hat_list[i],
        B_hat_list[i],
        well_formed=wf,
    )

# save the trial summary
write_qme_trial_summary(nc, a_err, B_err, well_formed=wf)

In [ ]:
# get the worst_a, worst_B (showing nc=3, wf=True)
idx_max = np.argmax(B_err)
worst_tid = trial_ids_out[idx_max]
worst_a = a_list[worst_tid]
worst_B = B_list[worst_tid]

In [ ]:
oracle = Oracle(worst_a, worst_B)
# check_i checks for gradient inconsistencies; looks at hidden oracle metric to check as it goes
qm = QME(sphere, oracle, search_tol, check_i=True)
ahat_worst, Bhat_worst = qm.run_qme()

In [ ]:
worst_a

In [ ]:
ahat_worst

In [ ]:
worst_B

In [ ]:
Bhat_worst

In [ ]:
# no inconsistencies! this means every gradient was within 1e-3 cosine similarity to the true gradient
qm.inconsistencies

In [ ]:
# this again verifies that the true and calculated gradients were very close

print("f_z dist:", spatial.distance.cosine(qm.qmesc.f_z, qm.f_z_opt))
print("f_neg0 dist:", spatial.distance.cosine(qm.qmesc.f_neg0, qm.f_neg0_opt))

for idx in range(0, q):
    print(f"fs_{idx} dist:", spatial.distance.cosine(qm.qmesc.fs[idx], qm.fs_opt[idx]))

In [ ]:
# try running the algo with the true gradients (normalized)
f_z_opt = normalize(qm.f_z_opt)
f_neg0_opt = normalize(qm.f_neg0_opt)
fs_opt = [normalize(i) for i in qm.fs_opt]
# QMESC = QME Slope Calculator, calculates a, B given the slope estimates
# turn off well_formed because we want to see the algorithm run directly 
# without gradient clipping (see qme.py/clip_v for an explanation)
qmesc_opt = QMESC(sphere, 5e-3, f_z_opt, f_neg0_opt, fs_opt, well_formed=False)
ahat_worst_opt, Bhat_worst_opt = qmesc_opt.compute_a_b()

In [ ]:
worst_a

In [ ]:
# accurate!
ahat_worst_opt

In [ ]:
worst_B

In [ ]:
# accurate!
Bhat_worst_opt

In [ ]:
# original algo
print("a squared error:", np.linalg.norm(ahat_worst - worst_a))
print("B squared error:", np.linalg.norm(Bhat_worst - worst_B, ord='fro'))

In [ ]:
# algo with true grads
print("a squared error:", np.linalg.norm(ahat_worst_opt - worst_a))
print("B squared error:", np.linalg.norm(Bhat_worst_opt - worst_B, ord='fro'))

As we can see, even on the worst trial the optimal gradients were extremely close to the measured gradients. However, using the measured gradients still resulted in large error. **Thus, the error can be entirely attributed to the fact that fractions are not robust to error.** We know it is not code error because we use the same algorithm on the true gradients and get the right answer. Evidently, this bad output CAN happen for some random inputs $(a, B)$. The *well_formed* assumption mitigates this error (discussed in the paper, code - see *common.py/check_a_B_sphere_satisfy_conditions*, and in *explore_fractional_error.ipynb*)

Also, if you look at the plot for the error (below) you will see that it is severely right-tailed.

In [ ]:
plt.hist(B_err)